In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Type_2_Diabetes"
cohort = "GSE227080"

# Input paths
in_trait_dir = "../../input/GEO/Type_2_Diabetes"
in_cohort_dir = "../../input/GEO/Type_2_Diabetes/GSE227080"

# Output paths
out_data_file = "../../output/preprocess/Type_2_Diabetes/GSE227080.csv"
out_gene_data_file = "../../output/preprocess/Type_2_Diabetes/gene_data/GSE227080.csv"
out_clinical_data_file = "../../output/preprocess/Type_2_Diabetes/clinical_data/GSE227080.csv"
json_path = "../../output/preprocess/Type_2_Diabetes/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Early differentially expressed immunological genes   in mild and severe COVID-19"
!Series_summary	"We retrospectively analysed the expression of 579 immunological genes in 60 COVID-19 subjects (SARS +ve) and 59 COVID-negative (SARS -ve) subjects using the NanoString nCounter (Immunology panel), a technology based on multiplexed single-molecule counting. Biobanked Human peripheral blood mononuclear cells (PBMCs) samples underwent Nucleic Acid extraction and digital detection of mRNA to evaluate changes in antiviral gene expression between SARS -ve controls and patients with mild (SARS +ve Mild) and moderate/severe (SARS +ve Mod/Sev) disease."
!Series_overall_design	"119 samples (60 SARS-CoV-2 positive / 59 SARS-CoV-2 negative)"
Sample Characteristics Dictionary:
{0: ['gender: F', 'gender: M'], 1: ['age: 38', 'age: 66', 'age: 21', 'age: 29', 'age: 73', 'age: 35', 'age: 48', 'age: 70', 'age: 69', 'age: 31', 'age: 72', 'age: 41', 'age: 85', 'age: 79',

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the background information, this dataset is about the expression of 579 immunological genes
# using NanoString nCounter technology. This indicates gene expression data is available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we can see:
# - trait (Type_2_Diabetes): Not directly mentioned. The dataset is about COVID-19 severity, not diabetes.
# - age: Available in key 1
# - gender: Available in key 0

# Since this dataset is about COVID-19 and not Type_2_Diabetes, trait data is not available
trait_row = None
age_row = 1
gender_row = 0

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values to binary indicator."""
    # This function won't be used since trait_row is None, but we define it for completeness
    if not isinstance(value, str):
        return None
        
    # Extract the value after the colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # For Type_2_Diabetes, we would typically look for keywords indicating diabetes status
    # But this dataset doesn't contain diabetes information
    return None

def convert_age(value):
    """Convert age string to numeric value."""
    if not isinstance(value, str):
        return None
        
    # Extract the value after the colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
        
    # Extract the value after the colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is already defined from a previous step
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:", preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Clinical data (Type_2_Diabetes) not available. Skipping clinical feature extraction.")


Clinical data (Type_2_Diabetes) not available. Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ABCB1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'APP', 'ARG1', 'ARG2',
       'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5', 'ATG7', 'ATM', 'B2M',
       'B3GAT1', 'BATF', 'BATF3'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These are standard human gene symbols that don't require mapping
# ABCB1, ABL1, ADA, etc. are all recognized human gene symbols

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since there's no clinical data for Type_2_Diabetes in this dataset (as determined in Step 2),
# we cannot link clinical and genetic data or perform the subsequent analysis steps.

# 5. Conduct final quality validation and save the cohort information
# We set is_biased=True to indicate the dataset is not usable for this trait
is_trait_available = False
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=True,  # Set to True to indicate data is not usable
    df=normalized_gene_data,  # Use the gene data as our DataFrame
    note="Dataset contains gene expression data for COVID-19 severity, not Type_2_Diabetes."
)

print(f"Dataset usability validation completed. Is usable: {is_usable}")
print("This dataset does not contain Type_2_Diabetes information, so no linked data file was created.")

Normalized gene data saved to ../../output/preprocess/Type_2_Diabetes/gene_data/GSE227080.csv
Dataset usability validation completed. Is usable: False
This dataset does not contain Type_2_Diabetes information, so no linked data file was created.
